# CytoTable (convert) and Pandas (Merges) Performance Comparisons

This notebook explores CytoTable (convert) and Pandas (DataFrame merges) usage with datasets of varying size to help describe performance impacts.

In [1]:
import itertools
import json
import os
import pathlib
import re
import signal
import subprocess
from datetime import datetime

import pandas as pd
import plotly.express as px
import plotly.io as pio
import psutil
from IPython.display import Image
from utilities import get_system_info

# set plotly default theme
pio.templates.default = "simple_white"

In [2]:
# show the system information
_ = get_system_info(show_output=True)


System Information:
Operating System: Darwin
Machine Type: arm64
Processor: arm
CPU Cores (Logical): 12
CPU Cores (Physical): 12
Total RAM (GB): 48.0
Python Version: 3.10.16


In [3]:
# observe the virtual env for dependency inheritance with memray
# from subprocedure calls
"/".join(
    subprocess.run(
        [
            "which",
            "memray",
        ],
        capture_output=True,
        check=True,
    )
    # decode bytestring as utf-8
    .stdout.decode("utf-8")
    # remove personal file structure
    .split("/")[6:]
    # replace final newline
).replace("\n", "")

'virtualenvs/cytotable-benchmarks-t7y5cEfp-py3.10/bin/memray'

In [4]:
# target file or table names
image_dir = "images"
examples_dir = "examples"
join_read_time_image = (
    f"{image_dir}/cytotable-and-pandas-comparisons-join-completion-time-csv.png"
)
join_mem_size_image = (
    f"{image_dir}/cytotable-and-pandas-comparisons-join-memory-size-csv.png"
)
example_files_list = [
    f"{examples_dir}/cytotable_convert_examplehuman_multiprocess_csv.py",
    f"{examples_dir}/cytotable_convert_examplehuman_multithread_csv.py",
    f"{examples_dir}/pandas_merge_examplehuman_csv.py",
]
example_data_list = [
    f"{examples_dir}/data/examplehuman_cellprofiler_features_csv",
    f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x2",
    # f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x4",
    # f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x8",
    # f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x16",
    # f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x32",
    # f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x64",
    # f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x128",
    # f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x256",
    # f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x512",
    # f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x1024",
    # f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x2048",
    # f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x4096",
]

# format for memray time strings
tformat = "%Y-%m-%d %H:%M:%S.%f%z"

In [5]:
# avoid a "cold start" for tested packages by using them before benchmarks
for example_file in example_files_list:
    _ = subprocess.run(
        ["python", example_file, example_data_list[0]],
        check=True,
    )

In [6]:
# Define the Parquet file path
results_parquet_file = "cytotable_pandas_results.parquet"

# Load existing results if available
if pathlib.Path(results_parquet_file).exists():
    df_results = pd.read_parquet(results_parquet_file)
    results = df_results.to_dict(orient="records")
else:
    results = []

In [7]:
# Number of iterations for each combination
num_iterations = 6

# Loop through each combination of example file and data file
for example_file, example_data in itertools.product(
    example_files_list, example_data_list
):
    for iteration in range(num_iterations):
        # Skip if this combination and iteration are already processed
        if any(
            result["file_input"] == example_file
            and result["data_input"] == example_data
            and result["iteration"] == iteration
            for result in results
        ):
            print(
                f"Skipping already processed: {example_file} with {example_data}, iteration {iteration}"
            )
            continue

        target_bin = f"{example_file}_with_{example_data.replace(f'{examples_dir}/data/', '')}_iter_{iteration}.memray.bin"
        target_json = f"{target_bin}.json"

        try:
            # Run the example file with memray
            subprocess.run(
                [
                    "memray",
                    "run",
                    "--follow-fork",
                    "--output",
                    target_bin,
                    "--force",
                    example_file,
                    example_data,
                ],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
                check=True,
            )

            # Generate memray stats
            subprocess.run(
                [
                    "memray",
                    "stats",
                    "--json",
                    "--output",
                    target_json,
                    "--force",
                    target_bin,
                ],
                capture_output=True,
                check=True,
            )

            # Load the JSON data
            with open(target_json) as memray_json_file:
                memray_data = json.load(memray_json_file)

            # Append the result
            results.append(
                {
                    "file_input": example_file.replace(f"{examples_dir}/", ""),
                    "data_input": example_data,
                    "iteration": iteration,
                    "time_duration (secs)": (
                        datetime.strptime(memray_data["metadata"]["end_time"], tformat)
                        - datetime.strptime(
                            memray_data["metadata"]["start_time"], tformat
                        )
                    ).total_seconds(),
                    "total_memory (bytes)": memray_data["total_bytes_allocated"],
                }
            )

            # Save intermediate results to Parquet
            df_results = pd.DataFrame(results)
            df_results.to_parquet(results_parquet_file, index=False)

        except Exception as e:
            print(
                f"Error processing {example_file} with {example_data}, iteration {iteration}: {e}"
            )

        finally:
            # Cleanup temporary files
            pathlib.Path(target_bin).unlink(missing_ok=True)
            pathlib.Path(target_json).unlink(missing_ok=True)
            print(
                f"Finished {example_file} with {example_data}, iteration {iteration}."
            )


# Final save to Parquet
df_results = pd.DataFrame(results)
df_results.to_parquet(results_parquet_file, index=False)

print(f"Processing complete. Results saved to '{results_parquet_file}'.")

Finished examples/cytotable_convert_examplehuman_multiprocess_csv.py with examples/data/examplehuman_cellprofiler_features_csv, iteration 0.
Finished examples/cytotable_convert_examplehuman_multiprocess_csv.py with examples/data/examplehuman_cellprofiler_features_csv, iteration 1.
Finished examples/cytotable_convert_examplehuman_multiprocess_csv.py with examples/data/examplehuman_cellprofiler_features_csv-x2, iteration 0.
Finished examples/cytotable_convert_examplehuman_multiprocess_csv.py with examples/data/examplehuman_cellprofiler_features_csv-x2, iteration 1.
Finished examples/cytotable_convert_examplehuman_multithread_csv.py with examples/data/examplehuman_cellprofiler_features_csv, iteration 0.
Finished examples/cytotable_convert_examplehuman_multithread_csv.py with examples/data/examplehuman_cellprofiler_features_csv, iteration 1.
Finished examples/cytotable_convert_examplehuman_multithread_csv.py with examples/data/examplehuman_cellprofiler_features_csv-x2, iteration 0.
Finishe

In [8]:
# add columns for data understandability in plots
def get_file_size_mb(path):
    """
    Returns the size in MB of a file or total size of all files in a directory.
    """
    p = pathlib.Path(path)
    try:
        if p.is_file():
            return p.stat().st_size / 1024 / 1024
        elif p.is_dir():
            return (
                sum(f.stat().st_size for f in p.rglob("*") if f.is_file()) / 1024 / 1024
            )
        else:
            return None
    except FileNotFoundError:
        return None


# memory usage in MB
df_results["total_memory (GB)"] = (
    df_results["total_memory (bytes)"] / 1024 / 1024 / 1024
)

# data input size additions for greater context
df_results["data_input_size_mb"] = df_results["data_input"].apply(get_file_size_mb)
df_results["data_input_with_size"] = (
    df_results["data_input"]
    + " ("
    + round(df_results["data_input_size_mb"]).astype("int64").astype("str")
    + " MB)"
)

# rename data input to simplify
df_results["data_input_renamed"] = (
    df_results["data_input_with_size"]
    .str.replace(f"{examples_dir}/data/", "")
    .str.replace("examplehuman_cellprofiler_features_", "input_")
)
df_results

,file_input,data_input,iteration,time_duration (secs),total_memory (bytes),total_memory (GB),data_input_size_mb,data_input_with_size,data_input_renamed
0,cytotable_convert_examplehuman_multiprocess_cs...,examples/data/examplehuman_cellprofiler_featur...,0,12.829,251280289,0.234023,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB)
1,cytotable_convert_examplehuman_multiprocess_cs...,examples/data/examplehuman_cellprofiler_featur...,1,13.592,251282609,0.234025,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB)
2,cytotable_convert_examplehuman_multiprocess_cs...,examples/data/examplehuman_cellprofiler_featur...,0,13.838,284774679,0.265217,2.673804,examples/data/examplehuman_cellprofiler_featur...,input_csv-x2 (3 MB)
3,cytotable_convert_examplehuman_multiprocess_cs...,examples/data/examplehuman_cellprofiler_featur...,1,13.903,251228974,0.233975,2.673804,examples/data/examplehuman_cellprofiler_featur...,input_csv-x2 (3 MB)
4,cytotable_convert_examplehuman_multithread_csv.py,examples/data/examplehuman_cellprofiler_featur...,0,3.025,15305606725,14.254457,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB)
5,cytotable_convert_examplehuman_multithread_csv.py,examples/data/examplehuman_cellprofiler_featur...,1,2.981,15307076460,14.255826,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB)
6,cytotable_convert_examplehuman_multithread_csv.py,examples/data/examplehuman_cellprofiler_featur...,0,3.401,15314179623,14.262441,2.673804,examples/data/examplehuman_cellprofiler_featur...,input_csv-x2 (3 MB)
7,cytotable_convert_examplehuman_multithread_csv.py,examples/data/examplehuman_cellprofiler_featur...,1,3.146,15315409659,14.263587,2.673804,examples/data/examplehuman_cellprofiler_featur...,input_csv-x2 (3 MB)
8,pandas_merge_examplehuman_csv.py,examples/data/examplehuman_cellprofiler_featur...,0,0.830,314075957,0.292506,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB)
9,pandas_merge_examplehuman_csv.py,examples/data/examplehuman_cellprofiler_featur...,1,1.014,314075957,0.292506,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB)


In [9]:
# build cols for split reference in the plot
df_results["cytotable_time_duration (multiprocess) (secs)"] = df_results[
    df_results["file_input"] == "cytotable_convert_examplehuman_multiprocess_csv.py"
]["time_duration (secs)"]
df_results["cytotable_total_memory (multiprocess) (GB)"] = df_results[
    df_results["file_input"] == "cytotable_convert_examplehuman_multiprocess_csv.py"
]["total_memory (GB)"]
df_results["cytotable_time_duration (multithread) (secs)"] = df_results[
    df_results["file_input"] == "cytotable_convert_examplehuman_multithread_csv.py"
]["time_duration (secs)"]
df_results["cytotable_total_memory (multithread) (GB)"] = df_results[
    df_results["file_input"] == "cytotable_convert_examplehuman_multithread_csv.py"
]["total_memory (GB)"]
df_results["pandas_time_duration (secs)"] = df_results[
    df_results["file_input"] == "pandas_merge_examplehuman_csv.py"
]["time_duration (secs)"]
df_results["pandas_total_memory (GB)"] = df_results[
    df_results["file_input"] == "pandas_merge_examplehuman_csv.py"
]["total_memory (GB)"]
df_results = (
    df_results.apply(lambda x: pd.Series(x.dropna().values))
    .drop(["file_input", "time_duration (secs)", "total_memory (GB)"], axis=1)
    .dropna()
)
df_results

,data_input,iteration,total_memory (bytes),data_input_size_mb,data_input_with_size,data_input_renamed,cytotable_time_duration (multiprocess) (secs),cytotable_total_memory (multiprocess) (GB),cytotable_time_duration (multithread) (secs),cytotable_total_memory (multithread) (GB),pandas_time_duration (secs),pandas_total_memory (GB)
0,examples/data/examplehuman_cellprofiler_featur...,0,251280289,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB),12.829,0.234023,3.025,14.254457,0.830,0.292506
1,examples/data/examplehuman_cellprofiler_featur...,1,251282609,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB),13.592,0.234025,2.981,14.255826,1.014,0.292506
2,examples/data/examplehuman_cellprofiler_featur...,0,284774679,2.673804,examples/data/examplehuman_cellprofiler_featur...,input_csv-x2 (3 MB),13.838,0.265217,3.401,14.262441,0.828,0.311071
3,examples/data/examplehuman_cellprofiler_featur...,1,251228974,2.673804,examples/data/examplehuman_cellprofiler_featur...,input_csv-x2 (3 MB),13.903,0.233975,3.146,14.263587,0.773,0.311073
4,examples/data/examplehuman_cellprofiler_featur...,0,15305606725,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB),11.534,0.265363,2.272,14.408664,0.648,0.292508
5,examples/data/examplehuman_cellprofiler_featur...,1,15307076460,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB),10.829,0.234064,2.183,14.224349,0.613,0.292508
6,examples/data/examplehuman_cellprofiler_featur...,0,15314179623,2.673804,examples/data/examplehuman_cellprofiler_featur...,input_csv-x2 (3 MB),11.792,0.265239,2.159,14.264712,0.627,0.311077
7,examples/data/examplehuman_cellprofiler_featur...,1,15315409659,2.673804,examples/data/examplehuman_cellprofiler_featur...,input_csv-x2 (3 MB),11.501,0.359728,2.168,14.260850,0.610,0.311075


In [10]:
# Group by data_input_renamed and calculate mean, min, and max
aggregated_results = df_results.groupby("data_input_renamed").agg(
    {
        "cytotable_time_duration (multiprocess) (secs)": ["mean", "min", "max"],
        "cytotable_time_duration (multithread) (secs)": ["mean", "min", "max"],
        "pandas_time_duration (secs)": ["mean", "min", "max"],
        "cytotable_total_memory (multiprocess) (GB)": ["mean", "min", "max"],
        "cytotable_total_memory (multithread) (GB)": ["mean", "min", "max"],
        "pandas_total_memory (GB)": ["mean", "min", "max"],
    }
)
# Flatten the multi-level columns
aggregated_results.columns = [
    f"{col[0]} ({col[1]})" for col in aggregated_results.columns
]
aggregated_results.reset_index(inplace=True)


# Helper function to extract numeric value or None
def sort_key(s):
    match = re.search(r"\d+", s)
    if match:
        return (1, int(match.group()))  # numeric items: (1, number)
    else:
        return (0, s.lower())  # non-numeric items: (0, alphabetical)


# Sort using the custom key
aggregated_results = aggregated_results.sort_values(
    by="data_input_renamed", key=lambda col: col.map(sort_key)
)
aggregated_results

,data_input_renamed,cytotable_time_duration (multiprocess) (secs) (mean),cytotable_time_duration (multiprocess) (secs) (min),cytotable_time_duration (multiprocess) (secs) (max),cytotable_time_duration (multithread) (secs) (mean),cytotable_time_duration (multithread) (secs) (min),cytotable_time_duration (multithread) (secs) (max),pandas_time_duration (secs) (mean),pandas_time_duration (secs) (min),pandas_time_duration (secs) (max),cytotable_total_memory (multiprocess) (GB) (mean),cytotable_total_memory (multiprocess) (GB) (min),cytotable_total_memory (multiprocess) (GB) (max),cytotable_total_memory (multithread) (GB) (mean),cytotable_total_memory (multithread) (GB) (min),cytotable_total_memory (multithread) (GB) (max),pandas_total_memory (GB) (mean),pandas_total_memory (GB) (min),pandas_total_memory (GB) (max)
0,input_csv (1 MB),12.1960,10.829,13.592,2.61525,2.183,3.025,0.77625,0.613,1.014,0.241869,0.234023,0.265363,14.285824,14.224349,14.408664,0.292507,0.292506,0.292508
1,input_csv-x2 (3 MB),12.7585,11.501,13.903,2.71850,2.159,3.401,0.70950,0.610,0.828,0.281040,0.233975,0.359728,14.262898,14.260850,14.264712,0.311074,0.311071,0.311077


In [11]:
# Time plot with min and max errors
fig = px.line(
    aggregated_results,
    x="data_input_renamed",
    y=[
        "cytotable_time_duration (multiprocess) (secs) (mean)",
        "cytotable_time_duration (multithread) (secs) (mean)",
        "pandas_time_duration (secs) (mean)",
    ],
    title="CytoTable and Pandas CSV Processing Time with Min/Max Errors",
    labels={"data_input_renamed": "Input File", "value": "Seconds"},
    height=500,
    width=900,
    symbol_sequence=["diamond"],
    color_discrete_sequence=[
        px.colors.qualitative.Vivid[6],
        px.colors.qualitative.Vivid[7],
        px.colors.qualitative.Vivid[4],
    ],
)

# Add error bars for each trace
for i, col in enumerate(
    [
        "cytotable_time_duration (multiprocess) (secs)",
        "cytotable_time_duration (multithread) (secs)",
        "pandas_time_duration (secs)",
    ]
):
    fig.data[i].update(
        error_y=dict(
            array=(
                aggregated_results[f"{col} (max)"] - aggregated_results[f"{col} (mean)"]
            ),
            arrayminus=(
                aggregated_results[f"{col} (mean)"] - aggregated_results[f"{col} (min)"]
            ),
        )
    )

# Rename the lines for the legend
newnames = {
    "cytotable_time_duration (multiprocess) (secs) (mean)": "CytoTable (multiprocess)",
    "cytotable_time_duration (multithread) (secs) (mean)": "CytoTable (multithread)",
    "pandas_time_duration (secs) (mean)": "Pandas",
}
fig.for_each_trace(
    lambda t: t.update(
        name=newnames[t.name],
        legendgroup=newnames[t.name],
        hovertemplate=t.hovertemplate.replace(t.name, newnames[t.name]),
    )
)

# Update the legend and layout
fig.update_layout(
    legend_title_text="",
    legend=dict(x=0.01, y=0.98, bgcolor="rgba(255,255,255,0.8)"),
    font=dict(size=16),  # Global font size
)
fig.update_traces(mode="lines+markers")
fig.update_layout(yaxis=dict(rangemode="tozero", autorange=True))

# Save and display the plot
fig.write_image(join_read_time_image)
fig.write_image(join_read_time_image.replace(".png", ".svg"))
Image(url=join_read_time_image.replace(".png", ".svg"))

In [12]:
# Memory plot with min and max errors
fig = px.line(
    aggregated_results,
    x="data_input_renamed",
    y=[
        "cytotable_total_memory (multiprocess) (GB) (mean)",
        "cytotable_total_memory (multithread) (GB) (mean)",
        "pandas_total_memory (GB) (mean)",
    ],
    title="CytoTable and Pandas CSV Total Memory Consumption with Min/Max Errors",
    labels={"data_input_renamed": "Input File", "value": "Gigabytes (GB)"},
    height=500,
    width=900,
    symbol_sequence=["diamond"],
    color_discrete_sequence=[
        px.colors.qualitative.Vivid[6],
        px.colors.qualitative.Vivid[7],
        px.colors.qualitative.Vivid[4],
    ],
)

# Add error bars for each trace
for i, col in enumerate(
    [
        "cytotable_total_memory (multiprocess) (GB)",
        "cytotable_total_memory (multithread) (GB)",
        "pandas_total_memory (GB)",
    ]
):
    fig.data[i].update(
        error_y=dict(
            array=(
                aggregated_results[f"{col} (max)"] - aggregated_results[f"{col} (mean)"]
            ),
            arrayminus=(
                aggregated_results[f"{col} (mean)"] - aggregated_results[f"{col} (min)"]
            ),
        )
    )

# Rename the lines for the legend
newnames = {
    "cytotable_total_memory (multiprocess) (GB) (mean)": "CytoTable (multiprocess)",
    "cytotable_total_memory (multithread) (GB) (mean)": "CytoTable (multithread)",
    "pandas_total_memory (GB) (mean)": "Pandas",
}
fig.for_each_trace(
    lambda t: t.update(
        name=newnames[t.name],
        legendgroup=newnames[t.name],
        hovertemplate=t.hovertemplate.replace(t.name, newnames[t.name]),
    )
)

# Update the legend and layout
fig.update_layout(
    legend_title_text="",
    legend=dict(x=0.01, y=0.98, bgcolor="rgba(255,255,255,0.8)"),
    font=dict(size=16),  # Global font size
)
fig.update_traces(mode="lines+markers")
fig.update_layout(yaxis=dict(rangemode="tozero", autorange=True))

# Save and display the plot
fig.write_image(join_mem_size_image)
fig.write_image(join_mem_size_image.replace(".png", ".svg"))
Image(url=join_mem_size_image.replace(".png", ".svg"))